In [6]:
#配上这里的解说：https://blog.csdn.net/lijin6249/article/details/78955175
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
# 创建输入数据
#Xy有两条数据，每条数据有四个时间步，每个时间步输入是长度为5的向量
X = np.random.randn(2, 4, 5)# 批次 、序列长度、样本维度
print(X)

# 第二个样本长度为3  将X的第二条数据的第三步及以后都置零，这步手动将第二条数据的
#步数缩短为2步。（以便后面体现dynamicrnn的便利）
X[1,2:] = 0
print(X)

#这步是定义每条数据的时间步数（第一条有四步，第二条有两步）
seq_lengths = [4, 2]

Gstacked_rnn = []
Gstacked_bw_rnn = []
#这里用了三层的gru，如果用lstm也是一样的套路
for i in range(3):
    
    #下面两句里的三是本层的单元数是3，这样输出长度就为3的向量。
    Gstacked_rnn.append(tf.contrib.rnn.GRUCell(3))
    Gstacked_bw_rnn.append(tf.contrib.rnn.GRUCell(3))

#建立前向和后向的三层RNN
Gmcell = tf.contrib.rnn.MultiRNNCell(Gstacked_rnn)
Gmcell_bw = tf.contrib.rnn.MultiRNNCell(Gstacked_bw_rnn)

sGbioutputs, sGoutput_state_fw, sGoutput_state_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn([Gmcell],[Gmcell_bw], X,sequence_length=seq_lengths,                                           dtype=tf.float64)
print(sGoutput_state_fw[0])

Gbioutputs, Goutput_state_fw = tf.nn.bidirectional_dynamic_rnn(Gmcell,Gmcell_bw, X,sequence_length=seq_lengths,dtype=tf.float64)

[[[-1.06330002 -1.6277488   0.67079667  0.21676019  1.54867189]
  [-1.3921073   2.32283792 -0.00250361 -0.53558677 -0.0083708 ]
  [-0.01235162 -0.2523287  -0.44393712  1.15784997  0.75911585]
  [-0.44185104 -1.3685178   1.36309256  0.06392272  1.06532389]]

 [[ 0.79846544 -0.93057389  0.38052451 -1.00611644  0.49802966]
  [ 0.77160553 -0.77193227  0.07702499 -0.39878322  0.46275217]
  [-1.38551485 -1.3315589  -0.59122004  1.18298241  0.43516593]
  [-0.33603246 -0.55673033  1.97620992 -0.04323674  0.94787764]]]
[[[-1.06330002 -1.6277488   0.67079667  0.21676019  1.54867189]
  [-1.3921073   2.32283792 -0.00250361 -0.53558677 -0.0083708 ]
  [-0.01235162 -0.2523287  -0.44393712  1.15784997  0.75911585]
  [-0.44185104 -1.3685178   1.36309256  0.06392272  1.06532389]]

 [[ 0.79846544 -0.93057389  0.38052451 -1.00611644  0.49802966]
  [ 0.77160553 -0.77193227  0.07702499 -0.39878322  0.46275217]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.    

In [2]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

sgbresult,sgstate_fw,sgstate_bw=sess.run([sGbioutputs,sGoutput_state_fw,sGoutput_state_bw])
print("全序列：\n", sgbresult[0])
print("短序列：\n", sgbresult[1])
print('Gru的状态：',len(sgstate_fw[0]),'\n',sgstate_fw[0][0],'\n',sgstate_fw[0][1],'\n',sgstate_fw[0][2])
print('Gru的状态：',len(sgstate_bw[0]),'\n',sgstate_bw[0][0],'\n',sgstate_bw[0][1],'\n',sgstate_bw[0][2])

#对下面输出的解释：首先
#全序列：写的人起了个破名字，其实这就是第一条数据的输出，第二条数据因为步长手动减少了，
#所以相对来说是短序列。 sgbresult输出的是[batch_site,max_time,layers_output]
#sgbresult[0]也就是第一条输出，本输出总共有四行，是因为第一条数据有4步，每步输出的6个值，
#前三个值是前向网络的输出（三是因为上面网络中gru单元输出定shape的三），后三个是后向的

#短序列同理

#sgstate_fw,即函数的第二个输出，是前向的final_state,包括h和c，sgstate_fw[0]表示h
#h也是一个元组，sgstate_fw[0]包括了三层的状态，也就是sgstate_fw[0][0],sgstate_fw[0][1]
#sgstate_fw[0][2], 同时，可以看到，sgstate_fw[0][0]长度为6，是两条数据，每条最终状态都是
#3造成的。

#而反向可能会让人迷惑，其实只要明白，反向网络的最终状态是第一个时间步（因为反着输入的嘛）
#sgstate_bw[0](也就是反向的h)同样是包括了三个层的状态，拿最后一层的状态sgstate_bw[2]为
#例，这其实就是第一个时间步的输出，输出值长度为6，就是第一个时间步两条数据的各三条输出。

全序列：
 [[ 0.00186137 -0.00137263  0.00243812  0.00085383 -0.10283804  0.13501629]
 [ 0.00742559 -0.00421351  0.00804357  0.00374449 -0.07308348  0.09061855]
 [ 0.00998598 -0.00575115  0.01034144 -0.00125412 -0.0420605   0.04774949]
 [ 0.00855077 -0.0097446   0.00625111 -0.00110107 -0.01872834  0.0195598 ]]
短序列：
 [[ 0.00367557  0.00168543  0.00702262  0.03195176  0.02263612 -0.01436679]
 [ 0.00362736  0.00393701  0.00587138  0.00790239  0.01356343 -0.01141595]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]]
Gru的状态： 3 
 [[-0.10206916  0.34912521 -0.14471451]
 [ 0.07038607  0.04207276 -0.10285089]] 
 [[-0.05735781  0.00822549 -0.02484693]
 [-0.00205732 -0.00844095  0.00650623]] 
 [[ 0.00855077 -0.0097446   0.00625111]
 [ 0.00362736  0.00393701  0.00587138]]
Gru的状态： 3 
 [[-0.03204368  0.26579786  0.46941746]
 [-0.1922159   0.23100648 -0.0675489 ]] 
 [[ 0.0804005   0.21776849  0.11664888]
 